In [ ]:
import numpy as np




In [6]:
import numpy as np

def line_search_sqp(f, grad_f, h, jac_h, g, jac_g, hess_lagrangian=None, x0=None, tol=1e-6, max_iter=100, c1=1e-4, tau=0.5, use_bfgs=True):
    """
    Sequential Quadratic Programming (SQP) with line search.
    Parameters:
    f: Objective function
    grad_f: Gradient of the objective function
    h: Equality constraint function
    jac_h: Jacobian of the equality constraint function
    g: Inequality constraint function
    jac_g: Jacobian of the inequality constraint function
    hess_lagrangian: Hessian of the Lagrangian function (optional)
    x0: Initial guess
    tol: Tolerance for convergence
    max_iter: Maximum number of iterations
    c1: Wolfe condition parameter which is used to determine the step length
    tau: Step length reduction factor
    use_bfgs: Whether to use BFGS for Hessian approximation when True then the BFGS algorithm is used, when not then the user provided hessian is used
    Returns: x: Optimal solution
    """

    n = len(x0)
    x_k = np.array(x0)
    lambda_k = np.zeros_like(h(x_k))
    mu_k = np.zeros_like(g(x_k))
    B_k = np.eye(n) if use_bfgs else hess_lagrangian(x_k, lambda_k, mu_k)

    for k in range(max_iter):
        grad_L = grad_f(x_k) - jac_h(x_k).T @ lambda_k - jac_g(x_k).T @ mu_k

        try:
            p_k = -np.linalg.solve(B_k, grad_L)
        except np.linalg.LinAlgError:
            print("Hessian not invertible", k)
            break

        def merit(alpha):
            x_alpha = x_k + alpha * p_k
            return (
                f(x_alpha)
                + np.sum(np.abs(h(x_alpha)))
                + np.sum(np.abs(np.minimum(0, g(x_alpha))))
            )

        phi_0 = merit(0)
        phi_prime_0 = grad_f(x_k).T @ p_k \
                      - lambda_k.T @ np.abs(h(x_k)) \
                      - mu_k.T @ np.abs(np.minimum(0, g(x_k)))

        alpha = 1.0
        while merit(alpha) > phi_0 + c1 * alpha * phi_prime_0:
            alpha *= tau

        x_new = x_k + alpha * p_k
        lambda_new = lambda_k  # simplified
        mu_new = mu_k          # simplified

        if use_bfgs:
            s_k = x_new - x_k
            y_k = (grad_f(x_new) - jac_h(x_new).T @ lambda_new - jac_g(x_new).T @ mu_new) - grad_L

            if s_k.T @ y_k >= 0.2 * s_k.T @ B_k @ s_k:
                theta = 1.0
            else:
                theta = 0.8 * (s_k.T @ B_k @ s_k) / (s_k.T @ B_k @ s_k - s_k.T @ y_k)

            r_k = theta * y_k + (1 - theta) * (B_k @ s_k)
            B_k += np.outer(r_k, r_k) / (s_k.T @ r_k) - \
                   (B_k @ np.outer(s_k, s_k) @ B_k) / (s_k.T @ B_k @ s_k)

        if np.linalg.norm(p_k) < tol:
            break

        x_k = x_new
        lambda_k = lambda_new
        mu_k = mu_new

    x_opt = x_k
    f_val = f(x_opt)
    h_val = h(x_opt)
    g_val = g(x_opt)

    print("Optimal solution x* =", x_opt)
    print("Objective value f(x*) =", f_val)
    print("Equality constraint h(x*) =", h_val)
    print("Inequality constraint g(x*) =", g_val)

    return x_opt

In [ ]:
import numpy as np

# Himmelblau test function
def f(x):
    x1, x2 = x
    return (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2

def grad_f(x):
    x1, x2 = x
    df_dx1 = 4 * x1 * (x1**2 + x2 - 11) + 2 * (x1 + x2**2 - 7)
    df_dx2 = 2 * (x1**2 + x2 - 11) + 4 * x2 * (x1 + x2**2 - 7)
    return np.array([df_dx1, df_dx2])

# Constraints
def h(x): 
    return np.array([])

def jac_h(x):  # jac
    return np.empty((0, len(x)))

def g(x): 
    x1, x2 = x
    return np.array([
        (x1 + 2)**2 - x2,
        -4 * x1 + 10 * x2
    ])

def jac_g(x):
    x1, x2 = x
    return np.array([
        [2 * (x1 + 2), -1],
        [-4, 10]
    ])

# Call SQP algorithm
x0 = np.array([0.0, 0.0])
optimal_x = line_search_sqp(f, grad_f, h, jac_h, g, jac_g, x0=x0, tol=1e-6, max_iter=100, c1=1e-4, tau=0.5, use_bfgs=True)
optimal_x


Optimal solution x* = [2.99999984 2.00000059]
Objective value f(x*) = 4.915222144808569e-12
Equality constraint h(x*) = []
Inequality constraint g(x*) = [22.99999782  8.0000065 ]


array([2.99999984, 2.00000059])

In [10]:
def trust_region_sqp(
    f, grad_f, h, jac_h, g, jac_g,
    hess_lagrangian=None,
    x0=None,
    tol=1e-6, max_iter=100,
    delta0=1.0, delta_max=10.0,
    eta1=0.1, eta2=0.75,
    gamma_inc=2.0, gamma_dec=0.5,
    use_bfgs=True
):
    n = len(x0)
    x_k = np.array(x0)
    lambda_k = np.zeros_like(h(x_k))
    mu_k = np.zeros_like(g(x_k))
    B_k = np.eye(n) if use_bfgs else hess_lagrangian(x_k, lambda_k, mu_k)
    delta_k = delta0

    for k in range(max_iter):
        grad_L = grad_f(x_k) - jac_h(x_k).T @ lambda_k - jac_g(x_k).T @ mu_k

        try:
            p_full = -np.linalg.solve(B_k, grad_L)
        except np.linalg.LinAlgError:
            print("Hessian not invertible")
            break

        if np.linalg.norm(p_full) > delta_k:
            p_k = -delta_k * grad_L / np.linalg.norm(grad_L)
        else:
            p_k = p_full

        model_pred = f(x_k)
        model_val = f(x_k) + grad_f(x_k).T @ p_k + 0.5 * p_k.T @ B_k @ p_k
        f_new = f(x_k + p_k)

        rho_k = (f(x_k) - f_new) / (model_pred - model_val + 1e-10)  # avoid div by zero

        if rho_k > eta1:
            x_new = x_k + p_k
        else:
            x_new = x_k  # reject step

        if rho_k > eta2:
            delta_k = min(gamma_inc * delta_k, delta_max)
        elif rho_k < eta1:
            delta_k *= gamma_dec

        lambda_new = lambda_k
        mu_new = mu_k

        if use_bfgs and not np.array_equal(x_new, x_k):
            s_k = x_new - x_k
            y_k = grad_f(x_new) - jac_h(x_new).T @ lambda_new - jac_g(x_new).T @ mu_new - grad_L

            if s_k.T @ y_k >= 0.2 * s_k.T @ B_k @ s_k:
                theta = 1.0
            else:
                theta = 0.8 * (s_k.T @ B_k @ s_k) / (s_k.T @ B_k @ s_k - s_k.T @ y_k)

            r_k = theta * y_k + (1 - theta) * B_k @ s_k
            B_k += np.outer(r_k, r_k) / (s_k.T @ r_k + 1e-10) \
                 - (B_k @ np.outer(s_k, s_k) @ B_k) / (s_k.T @ B_k @ s_k + 1e-10)

        if np.linalg.norm(p_k) < tol:
            break

        x_k = x_new
        lambda_k = lambda_new
        mu_k = mu_new

    x_opt = x_k
    f_val = f(x_opt)
    h_val = h(x_opt)
    g_val = g(x_opt)

    print("Optimal solution x* =", x_opt)
    print("Objective value f(x*) =", f_val)
    print("Equality constraint h(x*) =", h_val)
    print("Inequality constraint g(x*) =", g_val)

    return x_opt


In [11]:
# Run Trust Region SQP on Himmelblau's test problem

trust_region_sqp(
    f=f,
    grad_f=grad_f,
    h=h,
    jac_h=jac_h,
    g=g,
    jac_g=jac_g,
    x0=np.array([0.0, 0.0]),
    tol=1e-6,
    max_iter=100,
    delta0=1.0,
    delta_max=5.0,
    eta1=0.1,
    eta2=0.75,
    gamma_inc=2.0,
    gamma_dec=0.5,
    use_bfgs=True
)


Optimal solution x* = [3. 2.]
Objective value f(x*) = 2.6448844383683955e-18
Equality constraint h(x*) = []
Inequality constraint g(x*) = [23.  8.]


array([3., 2.])

In [12]:
(3**2+2-11)**2 + (3+2**2-7)**2

0